# Génération de Données Synthétiques pour le Modèle de Sommeil

Ce notebook génère un jeu de données synthétique plus grand pour améliorer l'entraînement du modèle.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Définir un seed pour la reproductibilité
np.random.seed(42)

In [ ]:
# Nombre d'échantillons à générer
n_samples = 1000

# Générer des données avec des relations réalistes
data = {
    'Age': np.random.normal(35, 10, n_samples).clip(18, 80),
    'Gender': np.random.choice(['Male', 'Female'], n_samples),
    'Sleep Duration': np.random.normal(7, 1.5, n_samples).clip(4, 10),
    'Physical Activity Level': np.random.normal(45, 20, n_samples).clip(0, 120),
    'Stress Level': np.random.normal(5, 2, n_samples).clip(1, 10),
    'Heart Rate': np.random.normal(75, 10, n_samples).clip(60, 120),
    'Daily Steps': np.random.normal(8000, 3000, n_samples).clip(2000, 25000),
    'Blood Pressure': np.random.choice(['Normal', 'High'], n_samples, p=[0.7, 0.3])
}

df = pd.DataFrame(data)

# Créer une fonction pour calculer la qualité du sommeil de manière réaliste
def calculate_sleep_quality(row):
    base_quality = 7.0
    
    # Impact de la durée du sommeil (optimal autour de 7-8 heures)
    sleep_impact = -0.5 * abs(row['Sleep Duration'] - 7.5)
    
    # Impact du stress (plus de stress = moins bonne qualité)
    stress_impact = -0.3 * row['Stress Level']
    
    # Impact de l'activité physique (plus d'activité = meilleure qualité, jusqu'à un certain point)
    activity_impact = 0.02 * min(row['Physical Activity Level'], 60)
    
    # Impact des pas quotidiens
    steps_impact = 0.0003 * min(row['Daily Steps'] - 5000, 10000)
    
    # Impact de la fréquence cardiaque (optimal autour de 70)
    hr_impact = -0.02 * abs(row['Heart Rate'] - 70)
    
    # Calculer la qualité finale avec un peu de bruit aléatoire
    quality = base_quality + sleep_impact + stress_impact + activity_impact + steps_impact + hr_impact
    quality += np.random.normal(0, 0.5)
    
    return min(max(quality, 1), 10)

# Calculer la qualité du sommeil
df['Quality of Sleep'] = df.apply(calculate_sleep_quality, axis=1)

# Convertir les variables catégorielles
df['Gender_num'] = (df['Gender'] == 'Male').astype(int)
df['Blood_Pressure_num'] = (df['Blood Pressure'] == 'High').astype(int)

# Sauvegarder les données
df.to_csv('../data/sleep_data_synthetic.csv', index=False)
print(f"Données générées et sauvegardées avec {n_samples} échantillons")